In [ ]:
%use dataframe, kandy

In [ ]:
USE {
    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [ ]:
fun DataFrame<*>.addProjectColumn(): DataFrame<*> =
    add(this["benchmark"].map { (it as String).substringBefore('-') }.rename("project"))

val folder = "../output/base"

val smtData = DataFrame.readCSV("${folder}/smtData.csv").addProjectColumn()
val fullSmtData = DataFrame.read("${folder}/fullSmtData.csv").addProjectColumn()

println("smtData: ${smtData.columnNames()}")
println("fullSmtData: ${fullSmtData.columnNames()}")


In [ ]:
import org.jetbrains.letsPlot.scale.guideColorbar
import org.jetbrains.letsPlot.scale.guideLegend

smtData.plot {
    points {
        x(sat) { axis.name = "SAT" }
        y(unsat) { axis.name = "UNSAT" }
        color(sat.values.zip(unsat.values).map { (s, u) -> s < u }) {
            scale = categorical(
                false to Color.GREEN, true to Color.RED
            )
            legend {
                breaksLabeled(true to "SAT < UNSAT", false to "UNSAT > SAT")
            }
        }
    }
    abLine {
        slope.constant(1)
        intercept.constant(0)
        color = Color.ORANGE
        type = LineType.DASHED
    }
    layout {
        size = 800 to 400
    }
}

In [ ]:
smtData.plot { 
    val transform: (Number) -> Double = {
        log(it.toDouble() + 1, 5.0)
    }
    val s = sat.map { transform(it) }
    val u = unsat.map { transform(it) }
    
    points { 
        x(s) {
            axis {
                name = "SAT"
                breaksLabeled(0.0 to "0", 1.0 to "4", 2.0 to "24", 3.0 to "124")
            }
        }
        y(u) { 
            axis {
                name = "UNSAT"
                breaksLabeled(0.0 to "0", 1.0 to "4", 2.0 to "24", 3.0 to "124", 4.0 to "624")
            }
        }
        color(sat.values.zip(unsat.values).map { (s, u) -> s < u }) {
            scale = categorical(
                false to Color.GREEN, true to Color.RED
            )
            legend {
                breaksLabeled(true to "SAT < UNSAT", false to "UNSAT > SAT")
            }
        }
    }
    abLine {
        slope.constant(1)
        intercept.constant(0)
        color = Color.ORANGE
        type = LineType.DASHED
    }
    layout {
        size = 800 to 400
    }
//    facetWrap(2) {
//        facet(project)
//    }
}

In [ ]:
fun DataFrame<*>.filterUnsat(): DataFrame<*> = filter { get("result") == "UNSAT" }

val unsatSmtData = fullSmtData.filterUnsat()

In [ ]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale

fullSmtData.plot {
    boxplot(project, numberOfVariables)
    boxplot(numberOfVariables, xName = "ALL")
    y {
        axis.expand(additive = 0.5)
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
    layout {
        title = "Variables distribution"
        size = 800 to 400
    }
}

In [ ]:
unsatSmtData.plot {
    boxplot(project, ucNumberOfVariables)
    boxplot(ucNumberOfVariables, xName = "ALL")
    y {
        axis.expand(additive = 0.5)
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
    layout {
        title = "Variables distribution in unsat cores"
        size = 800 to 400
    }
}

In [ ]:
unsatSmtData.plot {
    boxplot(project, ucNumberOfClauses)
    boxplot(ucNumberOfClauses, xName = "ALL")
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
        axis.expand(additive = 0.5)
    }
    layout {
        title = "Clauses distribution in unsat cores"
        size = 800 to 400
    }
}

In [ ]:
fullSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(numberOfVariables) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Variables distribution"
            size = 800 to 400
        }
    }

In [ ]:
fullSmtData
//    .filter { result == "UNSAT" }
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(numberOfClauses) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Clauses distribution"
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(ucNumberOfVariables) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Variables distribution in unsat cores"
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
//    .filter { project == "GUAVA" }
    .plot {
        statDensity(ucNumberOfClauses) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            title = "Clauses distribution in unsat cores"
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
    .filter { ucNumberOfVariables < 15 }
    .plot {
        statDensity(ucReps) {
            area {
                x(Stat.x)
                val d = Stat.density.cumSum()
                y(d.map { it / d.last() })
            }
        }
        x {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
        layout {
            size = 800 to 400
        }
    }

In [ ]:
unsatSmtData
//    .filter { project == "FASTJSON" }
    .plot {
        points {
            x(numberOfClauses)
            y(ucNumberOfVariables)
        }
//    x {
//        scale = PositionalContinuousScale<Double>(
//            null, null, null, Transformation.LOG2
//        )
//    }
    }

In [ ]:
fullSmtData.plot {
    boxplot(result, numberOfClauses)
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
}

In [ ]:
fullSmtData.plot {
    boxplot(result, numberOfVariables)
    y {
        scale = PositionalContinuousScale<Double>(
            null, null, null, Transformation.LOG2
        )
    }
}

In [ ]:
fullSmtData
    .filter { ucNumberOfVariables < 20 }
    .plot {
        boxplot(project, ucNumberOfClauses.mapIndexed { i, n -> numberOfClauses[i].toLong() * n }.rename("value"))
        y {
            scale = PositionalContinuousScale<Double>(
                null, null, null, Transformation.LOG2
            )
        }
    }